In [95]:
import requests
import pandas as pd
import random

from bs4 import BeautifulSoup

In [96]:
def scrape(url):

    response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36 OPR/94.0.0.0'})

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        return soup
    else:
        print(f"Failed to retrieve the page. Status code: {response.status_code}")

In [97]:
def get_max_page(url):
    soup = scrape(url)

    pages = soup.find('div', class_='pagination-links__container').find_all('a', class_='pagination-item')
    max_page = pages[-1].text.strip()

    return int(max_page)

In [98]:
def add_attribute(attribute):
    if attribute:
        return attribute.text.strip()
    return None

In [99]:
def get_product(url):
    soup = scrape(url)

    products = soup.find('div', class_='layout__Container-sc-nb1ebc-0 NDIcW')

    attributes = {'Name':[], 'Image Link':[], 'Origin':[], 'Price':[], 'Unit Price':[], 'Bio':[]}

    for product in products:
        name = product.find('a', class_='link__Link-sc-14ymsi2-0 hSRoYD link__Link-sc-14ymsi2-0 base__Title-sc-1mnb0pd-27 base__FixedHeightTitle-sc-1mnb0pd-43 hSRoYD ifdXVr cCRJZx')
        origin = product.find('span', class_='rectangular-badge__RectangularBadge-sc-1k9mcpf-0 base__CountryOfOrigin-sc-1mnb0pd-46 jPSXAR gYAJJo')
        price = product.find('strong', class_='base__Price-sc-1mnb0pd-29 sc-ksdxAp fJKWWj irZyf')
        unit_price = product.find('span', class_='text__Text-sc-6l1yjp-0 standard-promotion__PromotionIntentText-sc-1vpsrpe-2 fop__PricePerText-sc-sgv9y1-5 dLNLFE fAoahS eNYENy')

        attributes['Name'].append(add_attribute(name))
        attributes['Origin'].append(add_attribute(origin))
        attributes['Price'].append(add_attribute(price))
        attributes['Unit Price'].append(add_attribute(unit_price))
        attributes['Image Link'].append(None)

        if name:
            if 'bio' in name.text.strip().lower():
                attributes['Bio'].append(True)
            else:
                attributes['Bio'].append(False)
        else:
            attributes['Bio'].append(None)

        attribute_df = pd.DataFrame(attributes) 
        attribute_df.dropna(how = 'all', inplace=True)

    #multi_attributes = attribute_df.set_index(['Bio'])

    return attribute_df

In [100]:
urls = ['https://plus.casino.fr/products?source=navigation&sublocationId=920fdaca-fbb0-4f0a-8969-1d4aaa590359']

for url in urls:
    retail_df = get_product(url)

In [104]:
retail_df[retail_df['Bio'] == True]

,Name,Image Link,Origin,Price,Unit Price,Bio
11,Max Havelaar Banane Bio x5,None,Pays d’origine Équateur,"1,99 €","(0,40 € / article)",True
19,Concombre Bio 1 unité,None,Pays d’origine Espagne,"1,99 €","(1,99 € / article)",True
26,Bouquet Bio Tomates cerise Bio 250g,None,Pays d’origine Italie,"2,99 €","(11,96 € / kilo)",True
28,Casino Banane Bio Max Havelaar 5 fruits C2,None,Pays d’origine République dominicaine,"2,69 €","(3,84 € / kilo)",True
29,Bio Pomme bicolore x4,None,Pays d’origine France,"2,99 €","(0,75 € / article)",True
33,Ail Blanc Bio 3 têtes,None,Pays d’origine Espagne,"2,99 €","(1,00 € / article)",True
36,Casino Courgette Bio 1kg,None,Pays d’origine France,"5,49 €","(5,49 € / kilo)",True
45,Potimarron Bio X1,None,Pays d’origine France,"2,99 €","(2,99 € / article)",True
47,Bio Brocoli 500G,None,Pays d’origine Espagne,"3,99 €","(3,99 € / article)",True
49,Casino Bio Pomme Bicolore 4 Fruits C2,None,Pays d’origine France,"3,49 €","(2,68 € / kilo)",True


In [29]:
# Output the filtered dataframe to a CSV file
retail_df.to_csv('casino.csv', index=False)